# Addestramento modelli

In [1]:
import warnings
import numpy as np
import pandas as pd
import os
import sys
import json

In [2]:
warnings.filterwarnings("ignore")

In [3]:
import utility

In [4]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler, OneHotEncoder

In [5]:
import tensorflow as tf
from tensorflow import Tensor 
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dropout, Concatenate, Input, GlobalAveragePooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Attention, AdditiveAttention
from tensorflow.keras.layers import Activation, BatchNormalization, Add, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.regularizers import L1, L2, L1L2

In [6]:
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.models import load_model
from keras.layers.merge import concatenate

In [7]:
import kerastuner as kt
from kerastuner import Hyperband, BayesianOptimization, RandomSearch

In [8]:
K.set_epsilon(1)

In [9]:
model_trained_base_path = "TrainedModels"
dataset_path = os.path.join("..", "Preprocessing", "Dataset", "dataset_training.csv")

### Import dei dati

In [10]:
df = utility.import_dataset(dataset_path, "2021", "2023")
display(df)

,L1,L2,L3,SumOfLines,Temperature,DayOfWeek,Month,Day,Hour,Holiday
Time,,,,,,,,,,
2021-01-01 00:00:00,983.669142,1117.713692,961.125133,3062.507967,3.428421,4,1,1,0,1.0
2021-01-01 01:00:00,823.919029,1118.131796,930.218646,2872.269471,3.608938,4,1,1,1,1.0
2021-01-01 02:00:00,829.638000,1110.244854,989.942296,2929.825150,3.484612,4,1,1,2,1.0
2021-01-01 03:00:00,1012.239888,1104.282612,993.206479,3109.728979,3.456471,4,1,1,3,1.0
2021-01-01 04:00:00,812.574475,1142.363362,1002.239637,2957.177475,3.925238,4,1,1,4,1.0
...,...,...,...,...,...,...,...,...,...,...
2022-05-31 19:00:00,1758.702608,2371.511483,1678.265894,5808.479985,26.125096,1,5,31,19,0.0
2022-05-31 20:00:00,1634.401029,1945.340814,1664.862742,5244.604585,23.133508,1,5,31,20,0.0
2022-05-31 21:00:00,1742.510279,1890.756617,1589.791269,5223.058164,20.713387,1,5,31,21,0.0


### OneHot encoding e normalizzazione

In [11]:
df, scaler = utility.processing_dataset(df)

### Splitting in training, validation e test

In [12]:
df_train, df_validation, df_test = utility.split_dataset(df, "2021-03", "2022-04")

Train size: 9504 
Validation size: 1416 
Test size: 1464


### Creazione finestre

In [13]:
time_steps = 7 * 24
dim_other_feature = 24 + 7 + 12 + 1

In [14]:
train = utility.windowed_dataset(df_train, time_steps)
validation = utility.windowed_dataset(df_validation, time_steps)
test = utility.windowed_dataset(df_test, time_steps)

### Model 0 - TEST

In [15]:
def create_model_0():  

    # Parameters tuning
    time_steps = 7 * 24
    dim_other_feature = 24 + 7 + 12 + 1
    
    learning_rate = 0.01
    
    hp_dropout_rate = hp.Float("dropout_rate", min_value=0.1, max_value=0.5, step=0.1)
    
    regularizer_type = hp.Choice("regularizer_type", ["none", "L1", "L2"])
    regularizer_param = hp.Choice("regularizer_param", values=[1e-0, 1e-1, 1e-2, 1e-3], parent_name="regularizer_type", parent_values=["L1", "L2"])
    regularization_layer = hp.Choice("regularization_layer", ["all", "fc_only"], parent_name="regularizer_type", parent_values=["L1", "L2"])

    r_fc = None
    r_all = None
    
    if regularizer_type == "L1":
        regularizer = L1(regularizer_param)        
    elif regularizer_type == "L2":
        regularizer = L2(regularizer_param)
    elif regularizer_type == "none":
        regularizer = None  
        
    if regularization_layer == "all":
        r_fc = regularizer
        r_all = regularizer
    elif regularization_layer == "fc_only":
        r_fc = regularizer
    
    # Pipeline LOAD
    input_load = Input(shape=(time_steps, 1))
    c1 = Conv1D(32, kernel_size=3, activation="relu", kernel_regularizer=reg, bias_regularizer=reg)(input_load)
    m1 = MaxPooling1D(pool_size=2)(c1)
    c2 = Conv1D(64, kernel_size=3, activation="relu", kernel_regularizer=reg, bias_regularizer=reg)(m1)
    m2 = MaxPooling1D(pool_size=2)(c2)
    c3 = Conv1D(128, kernel_size=3, activation="relu", kernel_regularizer=reg, bias_regularizer=reg)(m2)
    flatten1 = Flatten()(c3)
    dropout1 = Dropout(rate=hp_dropout_rate)(flatten1)

    # Pipeline OTHER FEATURE
    input_other_feature = Input(shape=(dim_other_feature, ))
    dense1 = Dense(64, activation="relu", kernel_regularizer=reg, bias_regularizer=reg)(input_other_feature)
    dense2 = Dense(32, activation="relu", kernel_regularizer=reg, bias_regularizer=reg)(dense1)
    flatten2 = Flatten()(dense2)
    dropout2 = Dropout(rate=hp_dropout_rate)(flatten2)

    # Concatenate PIPLINES
    concatenated_layer = concatenate([dropout1,dropout2], axis=1)
    dense3 = Dense(units=1, activation="linear")(concatenated_layer)
    model = Model(inputs=[input_load, input_other_feature], outputs=dense3)
    
    # Compiling
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mse", metrics=["mape"])
    
    return model

### Model 1 - PAPER0 - LSTMwithoutOtherFeature

In [16]:
def create_model_1():

    # Parameters tuning
    time_steps = 7 * 24
    dim_other_feature = 24 + 7 + 12 + 1
    
    learning_rate = 0.01

    hp_dropout_rate = hp.Float("dropout_rate", min_value=0.1, max_value=0.5, step=0.1)
    
    regularizer_type = hp.Choice("regularizer_type", ["none", "L1", "L2"])
    regularizer_param = hp.Choice("regularizer_param", values=[1e-0, 1e-1, 1e-2, 1e-3], parent_name="regularizer_type", parent_values=["L1", "L2"])
    regularization_layer = hp.Choice("regularization_layer", ["all", "fc_only"], parent_name="regularizer_type", parent_values=["L1", "L2"])

    r_fc = None
    r_all = None
    
    if regularizer_type == "L1":
        regularizer = L1(regularizer_param)        
    elif regularizer_type == "L2":
        regularizer = L2(regularizer_param)
    elif regularizer_type == "none":
        regularizer = None  
        
    if regularization_layer == "all":
        r_fc = regularizer
        r_all = regularizer
    elif regularization_layer == "fc_only":
        r_fc = regularizer
    
    model = Sequential()
    model.add(Input(shape=(time_steps, 1)))
    model.add(LSTM(units=time_steps, return_sequences=True, kernel_regularizer=reg, bias_regularizer=reg))
    model.add(LSTM(units=time_steps, kernel_regularizer=reg, bias_regularizer=reg))
    model.add(Dropout(rate=hp_dropout_rate))
    model.add(Dense(units=1, activation="linear"))
    
    # Compiling
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mse", metrics=["mape"])
    
    return model

### Model 2 - PAPER12 - LSTMwithOtherFeature

In [17]:
def create_model_2(hp):
    
    # Parameters tuning
    time_steps = 7 * 24
    dim_other_feature = 24 + 7 + 12 + 1
    
    learning_rate = 0.01
    
    hp_dropout_rate = hp.Float("dropout_rate", min_value=0.1, max_value=0.5, step=0.1)
    
    regularizer_type = hp.Choice("regularizer_type", ["none", "L1", "L2"])
    regularizer_param = hp.Choice("regularizer_param", values=[1e-0, 1e-1, 1e-2, 1e-3], parent_name="regularizer_type", parent_values=["L1", "L2"])
    regularization_layer = hp.Choice("regularization_layer", ["all", "fc_only"], parent_name="regularizer_type", parent_values=["L1", "L2"])

    r_fc = None
    r_all = None
    
    if regularizer_type == "L1":
        regularizer = L1(regularizer_param)        
    elif regularizer_type == "L2":
        regularizer = L2(regularizer_param)
    elif regularizer_type == "none":
        regularizer = None  
        
    if regularization_layer == "all":
        r_fc = regularizer
        r_all = regularizer
    elif regularization_layer == "fc_only":
        r_fc = regularizer
    
    # Pipeline LOAD
    input_load = Input(shape=(time_steps, 1))
    lstm_layer = LSTM(time_steps, return_sequences=False, activity_regularizer=r_all)(input_load)
    dropout1 = Dropout(rate=hp_dropout_rate)(lstm_layer)

    # Pipeline OTHER FEATURE
    input_other_feature = Input(shape=(dim_other_feature, ))
    dense1 = Dense(64, activation="relu", activity_regularizer=r_fc)(input_other_feature)
    dropout2 = Dropout(rate=hp_dropout_rate)(dense1)

    # Concatenate PIPLINES
    concatenated_layer = concatenate([dropout1, dropout2], axis=1)
    dense2 = Dense(64, activation="relu", activity_regularizer=r_fc)(concatenated_layer)
    dropout3 = Dropout(rate=hp_dropout_rate)(dense2)
    output_layer = Dense(1, activation="linear", activity_regularizer=r_fc)(dropout3)
    model = Model(inputs=[input_load, input_other_feature], outputs=[output_layer])
    
    # Compiling
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mse", metrics=["mape"])
    
    return model

### Model 3 - PAPER13 - Conv1D + LSTM

In [18]:
def create_model_3(hp):

    # Parameters tuning
    time_steps = 7 * 24
    dim_other_feature = 24 + 7 + 12 + 1
    
    learning_rate = 0.01
    
    hp_dropout_rate = hp.Float("dropout_rate", min_value=0.1, max_value=0.5, step=0.1)
    
    regularizer_type = hp.Choice("regularizer_type", ["none", "L1", "L2"])
    regularizer_param = hp.Choice("regularizer_param", values=[1e-0, 1e-1, 1e-2, 1e-3], parent_name="regularizer_type", parent_values=["L1", "L2"])
    regularization_layer = hp.Choice("regularization_layer", ["all", "fc_only"], parent_name="regularizer_type", parent_values=["L1", "L2"])

    r_fc = None
    r_all = None
    
    if regularizer_type == "L1":
        regularizer = L1(regularizer_param)        
    elif regularizer_type == "L2":
        regularizer = L2(regularizer_param)
    elif regularizer_type == "none":
        regularizer = None  
        
    if regularization_layer == "all":
        r_fc = regularizer
        r_all = regularizer
    elif regularization_layer == "fc_only":
        r_fc = regularizer
    
    # Pipeline LOAD
    input_load = Input(shape=(time_steps, 1))

    # Pipeline 1
    branch1_1 = Conv1D(filters=64, kernel_size=3, activation="relu", activity_regularizer=r_all)(input_load)
    branch1_2 = MaxPooling1D(pool_size=2, strides=2)(branch1_1)

    # Pipeline 2
    branch2_1 = Conv1D(filters=64, kernel_size=5, activation="relu", activity_regularizer=r_all)(input_load)
    branch2_2 = MaxPooling1D(pool_size=2, strides=2)(branch2_1)

    # Pipeline 3
    branch3_1 = Conv1D(filters=64, kernel_size=7, activation="relu", activity_regularizer=r_all)(input_load)
    branch3_2 = MaxPooling1D(pool_size=2, strides=2)(branch3_1)

    # Concatenate pipelines LOAD
    concatenated_layer1 = concatenate([branch1_2,branch2_2,branch3_2], axis=1)
    dropout0 = Dropout(rate=hp_dropout_rate)(concatenated_layer1)
    lstm_layer = LSTM(64, return_sequences=True, activity_regularizer=r_all)(dropout0)
    global_average_pooling_layer = GlobalAveragePooling1D()(lstm_layer)

    # Pipeline OTHER FEATURE
    input_other_feature = Input(shape=(dim_other_feature, ))
    dense1 = Dense(64, activation="relu", activity_regularizer=r_fc)(input_other_feature)
    dropout1 = Dropout(rate=hp_dropout_rate)(dense1)
    flatten1 = Flatten()(dropout1)

    # Concatenate PIPLINES
    concatenated_layer2 = concatenate([global_average_pooling_layer,flatten1], axis=1)
    dense2 = Dense(64, activation="relu", activity_regularizer=r_fc)(concatenated_layer2)
    dropout2 = Dropout(rate=hp_dropout_rate)(dense2)
    dense3 = Dense(64, activation="relu", activity_regularizer=r_fc)(dropout2)
    dropout3 = Dropout(rate=hp_dropout_rate)(dense3)
    output_layer = Dense(1, activation="linear", activity_regularizer=r_fc)(dropout3)
    model = Model(inputs=[input_load, input_other_feature], outputs=[output_layer])
    
    # Compiling
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mse", metrics=["mape"])
    
    return model

### Model 4 - PAPER18 - Wavenet

In [19]:
def create_model_4(hp):
    
    # Parameters tuning
    time_steps = 7 * 24
    dim_other_feature = 24 + 7 + 12 + 1
    
    learning_rate = 0.01
    
    hp_dropout_rate = hp.Float("dropout_rate", min_value=0.1, max_value=0.5, step=0.1)
    
    regularizer_type = hp.Choice("regularizer_type", ["none", "L1", "L2"])
    regularizer_param = hp.Choice("regularizer_param", values=[1e-0, 1e-1, 1e-2, 1e-3], parent_name="regularizer_type", parent_values=["L1", "L2"])
    regularization_layer = hp.Choice("regularization_layer", ["all", "fc_only"], parent_name="regularizer_type", parent_values=["L1", "L2"])

    r_fc = None
    r_all = None
    
    if regularizer_type == "L1":
        regularizer = L1(regularizer_param)        
    elif regularizer_type == "L2":
        regularizer = L2(regularizer_param)
    elif regularizer_type == "none":
        regularizer = None  
        
    if regularization_layer == "all":
        r_fc = regularizer
        r_all = regularizer
    elif regularization_layer == "fc_only":
        r_fc = regularizer

    hp_n_neurons = hp.Int("n_neurons", min_value=8, max_value=64, step=8)
    
    # Pipeline LOAD
    input_load = Input(shape=(time_steps, 1))
    x = input_load
    for rate in (1, 2, 4, 8, 16, 32):
        x = Conv1D(
            filters=32, kernel_size=2, strides=1, padding="causal", activation="relu", dilation_rate=rate, 
            activity_regularizer=r_all
        )(x)
    flatten = Flatten()(x)

    # Pipeline OTHER FEATURE
    input_other_feature = Input(shape=(dim_other_feature, ))

    # Concatenate PIPLINES
    concatenated_layer = concatenate([flatten, input_other_feature], axis=1)
    dropout1 = Dropout(rate=hp_dropout_rate)(concatenated_layer)
    dense = Dense(hp_n_neurons, activation="relu", activity_regularizer=r_fc)(dropout1)
    dropout2 = Dropout(rate=hp_dropout_rate)(dense)
    output_layer = Dense(1, activation="linear", activity_regularizer=r_fc)(dropout2)
    model = Model(inputs=[input_load, input_other_feature], outputs=[output_layer])
    
    # Compiling
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mse", metrics=["mape"])
    
    return model

### Tuning modello

In [20]:
directory = "my_tuning_dir"
project_name = "model3"

In [21]:
tuner = BayesianOptimization(
    create_model_3,
    objective="val_mape",
    max_trials=15,
    directory=directory,
    project_name=project_name
)

In [22]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
dropout_rate (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
regularizer_type (Choice)
{'default': 'none', 'conditions': [], 'values': ['none', 'L1', 'L2'], 'ordered': False}
regularizer_param (Choice)
{'default': 1.0, 'conditions': [{'class_name': 'Parent', 'config': {'name': 'regularizer_type', 'values': ['L1', 'L2']}}], 'values': [1.0, 0.1, 0.01, 0.001], 'ordered': True}
regularization_layer (Choice)
{'default': 'all', 'conditions': [{'class_name': 'Parent', 'config': {'name': 'regularizer_type', 'values': ['L1', 'L2']}}], 'values': ['all', 'fc_only'], 'ordered': False}


In [23]:
early_stopping = EarlyStopping(monitor="val_mape", patience=10)

tuner.search(
    utility.get_x(train, "SumOfLines"), utility.get_y(train, "SumOfLines"), 
    epochs=50, 
    batch_size=256,
    validation_data=(utility.get_x(validation, "SumOfLines"), utility.get_y(validation, "SumOfLines")),
    callbacks=[early_stopping],
    shuffle=False
)

Trial 15 Complete [00h 07m 31s]
val_mape: 28.500810623168945

Best val_mape So Far: 26.503631591796875
Total elapsed time: 02h 07m 35s
INFO:tensorflow:Oracle triggered exit


In [24]:
with open(os.path.join(directory, project_name, "summary.txt"), "w") as file:
    original_stdout = sys.stdout
    sys.stdout = file
    output_funzione = tuner.results_summary()
    sys.stdout = original_stdout

In [25]:
with open(os.path.join(directory, project_name, "best_hyperparameters.txt"), "w") as file:
    json.dump(tuner.get_best_hyperparameters(num_trials=1)[0].values, file, indent=4)